In [51]:
data  = pd.read_excel('propDictionary.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['words','technique'])
print(data.head())

IndexError: list index out of range

# Import All Packages




In [1]:
    import random
    import pandas as pd
    import numpy as np
    import string
    from string import digits
    from sklearn.model_selection import train_test_split
    import torch
    import torch.nn as nn
    from sklearn.metrics import classification_report
    import transformers
    from transformers import AutoModel, BertTokenizerFast
    from ipywidgets import IntProgress
    
    from tqdm import tqdm
    import nltk
    import ssl

    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Import BERT Model, BERT Tokenizer and Torch

In [2]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# import Torch
device = torch.device("cpu")

# Predefined CLass

In [3]:
class Propaganda:
    NEGATIVE = 0
    POSITIVE = 1
class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = SUBJprop


class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_sentence(self):
        return [x.sentence for x  in self.reviews]

    def get_propaganda(self):
        return [int(x.propaganda) for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.propaganda == str(Propaganda.NEGATIVE), self.reviews))
        positive = list(filter(lambda x: x.propaganda == str(Propaganda.POSITIVE), self.reviews))
        negative_shrunk = negative[:len(positive)]
        self.reviews = positive + negative_shrunk
        random.shuffle(self.reviews)

# Split train dataset into train, validation and test sets

In [4]:
# step 2.1: Load Data
reviews = []
data  = pd.read_excel('Data/finalDataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['Sentence','SUBJprop'])
# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))


print("Total Rows:")
print(len(reviews))
print("Total Positive:")
print(len(list(filter(lambda x: x.propaganda == str(Propaganda.POSITIVE), reviews))))
print("Total Negative:")
print(len(list(filter(lambda x: x.propaganda == str(Propaganda.NEGATIVE), reviews))))

Total Rows:
14058
Total Positive:
3904
Total Negative:
10154


In [5]:
# step 2.2: Load Sentimental Data
SentimentWords= []
SentimentValue= []


Sentimentdata  = pd.read_excel('Data/SentimentalWords.xlsx', engine='openpyxl')
df = pd.DataFrame(Sentimentdata.astype(str) , columns = ['word','value'])

for index, row in df.iterrows():
    word = row['word']
    value = row['value']
    SentimentWords.append(word)
    SentimentValue.append(value)

In [45]:
neg_prop = list(filter(lambda x: x.propaganda == str(Propaganda.NEGATIVE), reviews))
pos_prop = list(filter(lambda x: x.propaganda == str(Propaganda.POSITIVE), reviews))
########################################################################################
#split trainig and DevTest dataset
neg_train, neg_devtest  = train_test_split(neg_prop , train_size=0.7, shuffle= False )
pos_train, pos_devtest = train_test_split(pos_prop , train_size=0.7, shuffle= False )
########################################################################################
#prepare training dataset
train = neg_train + pos_train
#random.shuffle(train)
########################################################################################
#prepare development and test dataset
neg_dev, neg_test = train_test_split(neg_devtest , train_size=0.5, shuffle= False )
pos_dev, pos_test = train_test_split(pos_devtest , train_size=0.5, shuffle= False )

dev = neg_dev + pos_dev
#random.shuffle(dev)

test = neg_test + pos_test
#random.shuffle(test)

In [46]:
pip install RandomUnderSampler


ERROR: Could not find a version that satisfies the requirement RandomUnderSampler (from versions: none)
ERROR: No matching distribution found for RandomUnderSampler
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
# step 3: Seperate the attribute, originally our array has text and score. we want them to be a seperate array
train_container = ReviewContainer(train)
train_container.evenly_distribute()

train_text = train_container.get_sentence()   
train_labels = train_container.get_propaganda() 

dev_text = [x.sentence for x in dev]
dev_labels = [int(x.propaganda) for x in dev]

test_text = [x.sentence for x in test]
test_labels = [int(x.propaganda) for x in test]



### Save splited Data to seprate excel files

In [9]:
# Create Excel file of train dataset
train_df = pd.DataFrame(train_text, columns=["Sentence"])
train_df['SUBJprop'] = train_labels
train_df.to_excel("./Data/trainDataset.xlsx")
# Create Excel file of dev dataset
dev_df = pd.DataFrame(dev_text, columns=["Sentence"])
dev_df['SUBJprop'] = dev_labels
dev_df['Tanbih'] = ""
dev_df.to_excel("./Data/devDataset.xlsx")
# Create Excel file of test dataset
test_df = pd.DataFrame(test_text, columns=["Sentence"])
test_df['SUBJprop'] = test_labels
test_df.to_excel("./Data/testDataset.xlsx")

In [26]:
max_seq_len = 100


# Tokenization and Filtering Punctuation

In [27]:
from nltk.tokenize import RegexpTokenizer
tokenizerNLTK = RegexpTokenizer(r'\w+')

train_text_tokenized = []
dev_text_tokenized = []
test_text_tokenized = []


for i in range(len(train_text)):
    train_text_tokenized.append(tokenizerNLTK.tokenize(train_text[i])) 

for i in range(len(dev_text)):
    dev_text_tokenized.append(tokenizerNLTK.tokenize(dev_text[i])) 

for i in range(len(test_text)):
    test_text_tokenized.append(tokenizerNLTK.tokenize(test_text[i])) 
    

print(test_text_tokenized[0:5])



[['WHO', 'said', 'there', 'should', 'not', 'be', 'restrictions', 'on', 'international', 'travel', 'or', 'trade'], ['In', 'the', 'meantime', 'it', 's', 'important', 'to', 'prevent', 'oneself', 'from', 'contracting', 'the', 'virus'], ['Unfortunately', 'health', 'care', 'workers', 'are', 'at', 'the', 'greatest', 'risk', 'as', 'they', 'try', 'to', 'cull', 'the', 'suffering', 'which', 'is', 'why', 'they', 'will', 'be', 'the', 'first', 'to', 'get', 'the', 'experimental', 'vaccine'], ['Pope', 'Francis', 'sexual', 'abuse', 'commission', 'hit', 'by', 'resignations', 'and', 'criticism', 'gets', 'a', 'reboot'], ['Last', 'week', 'Pope', 'Francis', 'announced', 'he', 'was', 'reviving', 'a', 'panel', 'he', 'created', 'to', 'advise', 'the', 'Vatican', 'on', 'how', 'to', 'handle', 'sexual', 'abuse', 'by', 'clergy']]


# Removing stop words

In [28]:
import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()


# Create list of word tokens after removing stopwords in train_text
train_text1=[]
for sentence in train_text_tokenized:
    temp=[]
    for word in sentence:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            temp.append(word) 
    train_text1.append(temp)
    
# print(train_text1[:100])
# print("*****************************************************")

# Create list of word tokens after removing stopwords in dev_text
dev_text1 =[] 

for sentence in dev_text_tokenized:
    temp=[]
    for word in sentence:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            temp.append(word) 
    dev_text1.append(temp)
    
# print(dev_text1[:100])
# print("*****************************************************")

# # Create list of word tokens after removing stopwords in test_text
test_text1 =[] 

for sentence in test_text_tokenized:
    temp=[]
    for word in sentence:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            temp.append(word) 
    test_text1.append(temp)
# print(test_text1[:100])  


# Repetition

In [ ]:
# train_text2=[]
# SentimentWords

# def checkIfDuplicates(listOfElems):
#     if len(listOfElems) == len(set(listOfElems)):
#         return False
#     else:
#         return True

# for sentence in train_text1:
#     temp=[]
#     if checkIfDuplicates(sentence):
#         print(sentence)



    # for word in sentence:
    #     temp.append(word)

    # train_text2.append(temp)

# print(train_text2[:10])

# Duplicate Sentimenal words

In [29]:
train_text2=[]

for sentence in train_text1:
    temp=[]
    for word in sentence:
        temp.append(word)
        for SentimentWord in SentimentWords:
            if word.lower()==SentimentWord:
                temp.append(word)

    train_text2.append(temp)

print(train_text2[:10])

[['added', 'office', 'authority', 'override', 'members', 'want', 'rule', 'breaker', 'government', 'network'], ['Ebola', 'haemorrhagic', 'fever', 'killed', '11', '000', 'world', 'decimated', 'West', 'Africa', 'spread', 'rapidly', 'space', 'years'], ['ways'], ['danger'], ['investigative', 'journalism', 'lifetime', 'equal', 'importance', 'WikiLeaks', 'calling', 'rapacious', 'power', 'account'], ['Kung', 'removed', 'priesthood', 'despite', 'manifest', 'heresy'], ['tells', 'Haig', 'said', 'Paddock'], ['0', '16', 'Employed', '127', '117', '128', '924', '1', '807', '1', '42', 'Employment', 'population', '59', '6', '60', '0', '0', '4', 'pts'], ['current', 'crisis', 'broader', 'goals', 'risk', 'White', 'House', 'scrambled', 'find', 'face', 'saving', 'solution', 'Saudi', 'rulers'], ['banned']]


# Detokenizer

In [30]:
train_text_detokenized=[]
for i in range(len(train_text1)):
    temp=""
    for j in range(len(train_text1[i])):
        temp=temp + " " + train_text1[i][j]        
    train_text_detokenized.append(temp)
# print(train_text_detokenized[:100])



dev_text_detokenized=[]
for i in range(len(dev_text1)):
    temp=""
    for j in range(len(dev_text1[i])):
        temp=temp + " " + dev_text1[i][j]        
    dev_text_detokenized.append(temp)
# print(dev_text_detokenized[:100])



test_text_detokenized=[]
for i in range(len(test_text1)):
    temp=""
    for j in range(len(test_text1[i])):
        temp=temp + " " + test_text1[i][j]        
    test_text_detokenized.append(temp)
print(test_text_detokenized[:100])

[' said restrictions international travel trade', ' meantime s important prevent oneself contracting virus', ' Unfortunately health care workers greatest risk try cull suffering experimental vaccine', ' Pope Francis sexual abuse commission hit resignations criticism gets reboot', ' week Pope Francis announced reviving panel created advise Vatican handle sexual abuse clergy', ' issue dogged Roman Catholic Church recent years critics accused Francis predecessors John Paul II Benedict XVI failing aggressively weed punish predator priests', ' s look drove panel s creation expect future', ' Pope Francis commission sexual abuse', ' Following criticism focusing halting child abuse church Francis created Pontifical Commission Protection Minors March 2014 naming Cardinal Sean O Malley archbishop Boston run', ' O Malley member pope s G9 group close advisors man sought clean abuse scandal Boston departure predecessor Cardinal Bernard Law shifting predator priests new posts exposed Boston Globe ch

In [42]:

# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text_detokenized,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    dev_text_detokenized,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text_detokenized,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
print(tokens_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': [[101, 2794, 2436, 3691, 2058, 15637, 2372, 2215, 3627, 24733, 2231, 2897, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1041, 24290, 5292, 6633, 2953, 25032, 22974, 2278, 9016, 2730, 2340, 2199, 2088, 11703, 20592, 2225, 3088, 3659, 5901, 2686, 2086, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 3971, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Convert Integer Sequences to Tensors

In [33]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels)

print(train_seq[0])

# for validation set
dev_seq = torch.tensor(tokens_val['input_ids'])
dev_mask = torch.tensor(tokens_val['attention_mask'])
dev_y = torch.tensor(dev_labels)

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels)


print(test_seq)
print(test_mask)
print(test_y)

tensor([  101,  2794,  2436,  3691,  2058, 15637,  2372,  2215,  3627, 24733,
         2231,  2897,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
tensor([[  101,  2056,  9259,  ...,     0,     0,     0],
        [  101, 12507,  1055,  ...,     0,     0,     0],
        [  101,  6854,  2740,  ...,     0,     0,     0],
        ...,
        [  101,  3218, 16263,  .

# Classifiers

## SVM

In [34]:

from sklearn import svm

clf_svm = svm.SVC(kernel='rbf',C=1, probability=True)
clf_svm.fit(train_seq, train_y)

clf_svm.predict(test_seq)
print(len(test_seq))


2110


## Decision Tree

In [35]:

from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_seq , train_y)

clf_dec.predict(test_seq)

array([1, 0, 1, ..., 0, 0, 0])

## Naive Bayes

In [36]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_seq , train_y)

clf_gnb.predict(test_seq)

array([0, 0, 0, ..., 0, 0, 0])

## Logistic Regression

In [43]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression(class_weight='balanced')
clf_log.fit(train_seq, train_y)

clf_log.predict(test_seq)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 0, 1, ..., 0, 0, 0])

# Evaluation

# F1

In [44]:
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(test_y, clf_svm.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Support Decision Tree
print(f1_score(test_y,clf_dec.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Support Naive Bayes
print(f1_score(test_y,clf_gnb.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

# For Logistic Regression
print(f1_score(test_y,clf_log.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


[0.39875389 0.73705722]
[0.39301875 0.64870932]
[0.28421053 0.79204893]
[0.37899219 0.68872287]


## Mean Accuracy

In [39]:
# For Support Vector Machine
print(clf_svm.score(test_seq,test_y))
# For Decision Tree
print(clf_dec.score(test_seq,test_y))
# For Decision Naive Bayes
print(clf_gnb.score(test_seq,test_y))
# For Logistic Regression
print(clf_log.score(test_seq,test_y))

0.6341232227488152
0.5549763033175356
0.6777251184834123
0.5853080568720379


## Precision

In [40]:
# Precision
from sklearn.metrics import precision_score

# For Support Vector Machine
print(precision_score(test_y, clf_svm.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Tree
print(precision_score(test_y, clf_dec.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Naive Bayes
print(precision_score(test_y, clf_gnb.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Logistic Regression
print(precision_score(test_y, clf_log.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

[0.36676218 0.76628895]
[0.31633715 0.75456919]
[0.37087912 0.7416953 ]
[0.32442284 0.75213675]


## Recall

In [41]:
from sklearn.metrics import recall_score

# For Support Vector Machine
print(recall_score(test_y, clf_svm.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Tree
print(recall_score(test_y, clf_dec.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Decision Naive Bayes
print(recall_score(test_y, clf_gnb.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
# For Logistic Regression
print(recall_score(test_y, clf_log.predict(test_seq),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

[0.43686007 0.70997375]
[0.51877133 0.56889764]
[0.23037543 0.84973753]
[0.4556314 0.6351706]


# VIP : Predict_Proba using Threshold

In [ ]:
AboveThresholdsvm = [] 
AboveThresholdsvmbrop = []
# [0][1] = Postivie
# [0][0] = Negative


for i in range(len(test_seq)):
  if clf_svm.predict_proba(test_seq[i].reshape(1, -1))[0][1]>0.60 or clf_svm.predict_proba(test_seq[i].reshape(1, -1))[0][0]>0.60:
    AboveThresholdsvm.append(test_text[i])
    AboveThresholdsvmbrop.append(test_labels[i])



print(len(AboveThresholdsvm))


In [ ]:
AboveThresholdLR = [] 
AboveThresholdLRbrop = []
# [0][1] = Postivie
# [0][0] = Negative


for i in range(len(test_seq)):
  if clf_log.predict_proba(test_seq[i].reshape(1, -1))[0][1]>0.60 or clf_log.predict_proba(test_seq[i].reshape(1, -1))[0][0]>0.60:
    AboveThresholdLR.append(test_text[i])
    AboveThresholdLRbrop.append(test_labels[i])



print(len(AboveThresholdsvm))

In [ ]:
# Bert
tokens_testsvm = tokenizer.batch_encode_plus(
    AboveThresholdsvm,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
test_seqsvm = torch.tensor(tokens_testsvm['input_ids'])
test_masksvm = torch.tensor(tokens_testsvm['attention_mask'])
test_ysvm = torch.tensor(AboveThresholdsvmbrop)

tokens_testLR = tokenizer.batch_encode_plus(
    AboveThresholdLR,
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)


test_seqLR = torch.tensor(tokens_testLR['input_ids'])
test_maskLR = torch.tensor(tokens_testLR['attention_mask'])
test_yLR = torch.tensor(AboveThresholdLRbrop)

## Evaluation

In [ ]:
#F1
print("F1 SVM more than 60% threshold")
print(f1_score(test_ysvm, clf_svm.predict(test_seqsvm),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

print("F1 LR more than 60% threshold")
print(f1_score(test_yLR, clf_log.predict(test_seqLR),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

In [ ]:
# Accuracy
# For Support Vector Machine
print("Accuracy SVM more than 60% threshold")
print(clf_svm.score(test_seqsvm,test_ysvm))
print("Accuracy LR more than 60% threshold")
print(clf_log.score(test_seqLR,test_yLR))

In [ ]:
# Precision
from sklearn.metrics import precision_score
# For Support Vector Machine
print("Precision SVM more than 60% threshold")
print(precision_score(test_ysvm, clf_svm.predict(test_seqsvm),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))
print("Precision LR more than 60% threshold")
print(precision_score(test_yLR, clf_log.predict(test_seqLR),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))


In [ ]:
# Recall
from sklearn.metrics import recall_score
print("Precision SVM more than 60% threshold")
print(recall_score(test_ysvm, clf_svm.predict(test_seqsvm),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))

print("Precision LR more than 60% threshold")
print(recall_score(test_yLR, clf_log.predict(test_seqLR),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEGATIVE]))